# 目标


根据需求，我们需要设计一个开机界面，并在界面上显示3个圆形按钮，按钮上需显示图片，界面大小800x500。界面上不显示最大化、最小化按钮

![](./image/qt_button_login.png)

# 知识点

* 按钮控件使用
* 信号与槽
* 设置按钮形状
* 按钮上显示图片

# QPushButton按钮控件

在温度显示中已经使用了QWidget控件和QLabel控件，并且提到过按钮空间。再次强调控件和窗体的区别，没有设定父控件的控件就是一个窗体。
>没有嵌入到其他控件中的控件称为窗体对象

QT的按钮控件有：
* 按钮类：QPushButton（普通按钮）、QToolButton（工具按钮，一般用在工具栏）、QRadioButton（单选框）、QCheckBox（复选框，有三种状态）

本文我们只研究普通按钮，重点研究按钮对事件的控制过程和按钮的显示效果，前者研究的是按钮的工作原理，后者研究的是如何提高用户体验。

**创建一个pushButton按钮**

使用之前先导入QPushButton, QToolButton, QRadioButton, QCheckButtion。

In [1]:
from PyQt5.QtWidgets import QApplication, QWidget,QPushButton
from PyQt5.QtCore import Qt

app = QApplication([])

为了把问题聚焦在按钮本身，此处我们先不创建widget窗体，而是把按钮作为窗口，因为按钮没有父对象

In [2]:
pushButton = QPushButton() # 创建按钮对象
pushButton.setText("pushButton") # 修改按钮上显示的文本信息
pushButton.show()

```cpp
//C++
QPushButton *pushButton = new QPushButton(this);//创建一个对象
pushButton->setText("pushButton");
```

In [ ]:
`QPushButton`函数是类`QPushButton`的实例化函数，这个函数有多个入口参数，即在创建的时候提供一些初始化参数。就像我们打电话预定餐厅位置的时候，同时点了一些菜品，这些菜品也可以订好后现场或再次打电话点菜，这里订餐位置我们可以理解为一个实例化的过程就像调用`QPushButton`函数，现场点菜就像后面的`setText`和`resize`函数。这里我们也可以调用`QPushButton`的时候直接指定text,就像`QPushButton("demoButton")`,然而`resize`却不能直接指定，就像有的菜品只能到店点一样。因为支持的越多函数会越复杂，会影响程序执行效率，所以只提供了最基本的功能。

`QPushButton`支持的有：按键文本、按键图标、按键的父控件（默认为空，即窗口）

In [3]:
demoButton = QPushButton("demoButton")
demoButton.show()

In [4]:
app.exec_()

0

 ```cpp
 //C++
    QPushButton *demoButton = new QPushButton(this);//再创建一个对象
    demoButton->setText("demoButton");
```

从程序的可读性考虑更推荐第一种方法。因为可以很清楚的看到不同属性的配置、修改过程。

这样就可以看到2个不同的按键了。

想象一下我们比较常见的按钮就是安装软件的时候，界面上出现的“确认”/“退出”。按下确认后会开始程序的安装、按下退出后直接关闭窗口退出软件。这是怎么做到的呢？要知道这里出先的按钮和我们手机上的音量按钮是一样的，不同的是手机上是用手指直接按的，这里的是用鼠标或子触摸屏上按的，但是两者的目的后原理都是相通的。手机上的按钮按下后会把2个金属片短接，**形成一个信号然后触发一个动作**，就像门铃一样按钮会触发铃声。

软件里的实现过程也是如此，鼠标按下按钮后会产生一个鼠标单击事件，qt系统是知道鼠标的按下动作是在某个按钮上发生的，这样qt就认为是按钮被按下了会发出一个信号，然后根据这个信号去执行一个对应的函数。也许这个过程用遥控器控制空调更好理解，遥控器发生信号，空调开机/关机，如果是同款的2台空调，那么2台将同时开机/关机，而不同品牌的则不会有任何动作。遥控器的这个例子很好的说明了qt中的信号与槽的使用，遥控器的信号发出是由按键触发的，无论是否有空调在都会发出，如果附近有与遥控器匹配的空调那么两者之间就存在一个绑定关系，空调将根据遥控器的信号执行不同动作，如果附近空调与遥控器不匹配，那么两者之间就不存在绑定关系，就不会有任何动作。

# 按钮控制灯的亮灭

我们设计一个圆形的标签，然后用按钮控制标签的不同颜色表示灯/LED的亮灭。这里主要是研究按钮对标签颜色控制，所以先用默认形状的标签开展，之后再研究标签形态的实现。

上面已经讲述了遥控器控制空调的例子，按钮实际上是用信号与槽机制实现操作控制对象的。

![](./image/qt_button_ctrl_led.png)

In [ ]:
from PyQt5.QtWidgets import QApplication, QWidget,QPushButton
from PyQt5.QtCore import Qt

app = QApplication([])

# 创建窗口用于放置按钮和标签
widgetMainWindow = QWidget()
widgetMainWindow.resize(300, 300)



```cpp
//C++
MainWindow w;
w.resize(800,300); // 设定尺寸
```

In [ ]:
from PyQt5.QtWidgets import QLabel

class MyLabel(QLabel):
    def __init__(self):
        QLabel.__init__(self)
        self.styleCurrent = 0
        self.styleDefault = "min-width: 16px; min-height: 16px; max-width: 16px; max-height: 16px; border-radius: 8px;  border: 1px solid black;"
        self.setStyleSheet(self.styleDefault + "background: red")
    
    def soltLabelStyleSheetUpdate(self):
        if self.styleCurrent == 0:
            self.setStyleSheet(self.styleDefault + "background: green")
            self.styleCurrent = 1
        else:
            self.setStyleSheet(self.styleDefault + "background: red")
            self.styleCurrent = 0

```cpp
//C++
#include "mylable.h"

MyLable::MyLable(QWidget *parent):QLabel(parent)//调用父类的构造函数
{
    //用一个私有的属性保存默认的风格
    this->styleDefault= "min-width: 16px; min-height: 16px; max-width: 16px; max-height: 16px; border-radius: 8px;  border: 1px solid black;";
    //设置风格
    this->setStyleSheet(this->styleDefault+"background: red");
}

//增加一个子类特有的方法
void MyLable::soltLabelStyleSheetUpdate()
{
    qDebug()<<tr("soltLabelStyleSheetUpdate!");
    //调整背景色
    if(this->styleCurrent){
        this->setStyleSheet(this->styleDefault+"background: red");
        this->styleCurrent = false;
    }else{
        this->setStyleSheet(this->styleDefault+"background: green");
        this->styleCurrent = true;
    }
}
```

这里我们创建了1个继承QLabel的标签类`MyLabel`，类中增加了一个槽函数`solt_label_style`用于实现标签状态的切换。继承是一个"is a"的关系，我们新创建的`MyLabel`继承了QLabel，所以它也是一个Label，并且拥有了QLabel的全部属性和功能，例如`setParent`函数`setGeometry`函数这两个函数本来就是QLabel从父类中继承过来的，现在MqLabel也拥有这些函数了。

`QLabel.__init__(self)`这个是继承中对父类初始化的显式调用，因为子类`MyQlabel`中定义了`__init__`函数。这个函数具体应用如下：
* 子类没有定义自己的初始化函数，父类的初始化函数会被默认调用；但是如果要实例化子类的对象，则只能传入父类的初始化函数对应的参数，否则会出错。
* 子类定义了自己的初始化函数，在子类中显示调用父类，子类和父类的属性都会被初始化。没有显示调用父类的初始化函数，则父类的属性不会被初始化。

也就是说如果执行`QLabel.__init__(self)`这个语句，那么用`setStyleSheet`这个函数是不能使用的，实例化的时候会提示：
```
RuntimeError: super-class __init__() of type MyLabel was never called
```

In [ ]:
ledLabel = MyLabel()
ledLabel.setParent(widgetMainWindow)
ledLabel.setGeometry(20, 20, 50, 50)

pushButton = QPushButton() 
pushButton.setParent(widgetMainWindow)
pushButton.clicked.connect(ledLabel.soltLabelStyleSheetUpdate)
pushButton.setGeometry(20, 70, 100, 20)

widgetMainWindow.show()

app.exec_()

```cpp
//C++
myLable->setGeometry(10, 10, 100, 50);//位置不对，设置正确位置
connect(pushButton,&QPushButton::clicked,this,&MainWindow::soltLabelStyleSheetUpdate);
pushButton->setGeometry(20, 70, 100, 20);
```

因为MyLabel继承了QLabel的所有属性和功能，所以可以直接使用`ledLabel.setGeometry，ledLabel.setParent`。如果不用继承方式实现，那么对这两个函数的引用将会麻烦一点。

In [1]:
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QLabel
from PyQt5.QtCore import Qt

app = QApplication([])

# 创建窗口用于放置按钮和标签
widgetMainWindow = QWidget()
widgetMainWindow.resize(300, 300)

# 没有父类
class MyLabel():
    def __init__(self):
#         QLabel.__init__(self)
        self.label = QLabel() # 需要创建标签
        self.styleCurrent = 0
        self.styleDefault = """
                            min-width: 16px; 
                            min-height: 16px;
                            max-width: 16px; 
                            max-height: 16px;
                            border-radius: 8px;  
                            border: 1px solid black;
                            """
        self.label.setStyleSheet(self.styleDefault + "background: red") # setStyleSheet函数不能直接引用
    
    def soltLabelStyleSheetUpdate(self):
        if self.styleCurrent == 0:
            self.label.setStyleSheet(self.styleDefault + "background: green")
            self.styleCurrent = 1
        else:
            self.label.setStyleSheet(self.styleDefault + "background: red")
            self.styleCurrent = 0
            
ledLabel = MyLabel()
ledLabel.label.setParent(widgetMainWindow) # setParent
ledLabel.label.setGeometry(20, 20, 50, 50) # setGeometry函数不能直接引用

pushButton = QPushButton() 
pushButton.setParent(widgetMainWindow)
pushButton.clicked.connect(ledLabel.soltLabelStyleSheetUpdate)
pushButton.setGeometry(20, 70, 100, 20)

widgetMainWindow.show()

app.exec_()

0

```cpp
//C++
#include "mylable.h"

MyLable::MyLable(QWidget *parent):QLabel(parent)//调用父类的构造函数
{
    //用一个私有的属性保存默认的风格
    this->styleDefault= "min-width: 16px; \
                            min-height: 16px;\
                            max-width: 16px; \
                            max-height: 16px;\
                            border-radius: 8px;  \
                            border: 1px solid black;min-width: 16px; \
                            min-height: 16px;\
                            max-width: 16px; \
                            max-height: 16px;\
                            border-radius: 8px;  \
                            border: 1px solid black;";
    //设置风格
    this->setStyleSheet(this->styleDefault+"background: red");
}

//增加一个子类特有的方法
void MyLable::soltLabelStyleSheetUpdate()
{
    qDebug()<<tr("soltLabelStyleSheetUpdate!");
    //调整背景色
    if(this->styleCurrent){
        this->setStyleSheet(this->styleDefault+"background: red");
        this->styleCurrent = false;
    }else{
        this->setStyleSheet(this->styleDefault+"background: green");
        this->styleCurrent = true;
    }
}
```

上面程序运行的结果与继承法实现的结果是一样的，不同点在于对子函数的引用上不同，上面没有使用继承，所以引用上必须通过`label`才能调用`setStyleSheet, setParent, setGeometry`函数。试想下，如果类之间的引用嵌套增多后，代码会变得很繁琐，且容易出错，继承很好的解决了这个问题。

`setStyleSheet`可以配置对象的背景颜色、字体颜色，字体类型等，不同类的样式表（`Qt Style Sheet`）也不完全相同。`setStyleSheet`实际修改的就是样式表，QT里的样式表很大程度受HTML中CSS的启发，把控件本身与控件的样式一定程度解耦，就像代码中的`self.styleDefault`定义的字符串的用途就是要配置标签空间的外观，代码中通过setStyleSheet完成了标签颜色切换。关于样式表的使用后续会更深入的探讨。

# 改变按钮样式

完成了按钮的对LED的控制操作后，我们来优化一下操作界面，首先是给按钮增加倒角，进一步控制按钮为圆形。无论是倒角、还是圆形、还是按钮颜色都属于按钮的显示风格属性，风格的控制通过`setStyleSheet`函数来设置样式表的方法实现。

## 圆形按钮

In [1]:
from PyQt5.QtWidgets import QApplication, QWidget,QPushButton
from PyQt5.QtCore import Qt

app = QApplication([])

# 创建窗口用于放置按钮和标签
widgetMainWindow = QWidget()
widgetMainWindow.resize(300, 300)

pushButton = QPushButton() 
pushButton.setParent(widgetMainWindow)
pushButton.setGeometry(20, 70, 100, 100)
# 增加属性配置表设置代码
# Todo
# pushButton.setStyleSheet("border-radius: 50; background-color: green")

widgetMainWindow.show()

app.exec_()

0

```cpp
//C++
w.resize(300,300);
QPushButon pushButton = new QPushButton(this);
pushButton.setGeometry(20, 70, 100, 100);
w.show();
```

1. 首先我们创建一个按钮，并设置按钮的几何尺寸为：100x100px。为了让按钮为圆形， 我们需要修改它的倒角参数,使倒角半径为50px，这个参数正是样式表的其中一个属性。所以代码行:

```
pushButton.setStyleSheet("border-radius: 50")
```

2. 但是实际显示效果是，界面上看不到按钮。这是因为，修改样式表的时候只配置了倒角半径参数，其他参数被设置为默认值，窗口对象`widget`使用的也是默认值，结果导致两者颜色相同而不能看到按钮了。需修改为：

```
pushButton.setStyleSheet("border-radius: 50; background-color: green")
```
3. 增加背景颜色,边框等属性：

```
pushButton.setStyleSheet("""
                 border-radius: 50; 
                 background-color: green;
                 border-width: 5;
                 border-color: white;
                 border-style: outset;
                 """)
                 
```

## 按钮的三态效果使用

按钮有三种状态：普通态、鼠标停留态、点击态。下面可以看一下按钮默认的三态效果。

In [1]:
from PyQt5.QtWidgets import QApplication, QWidget,QPushButton
from PyQt5.QtCore import Qt

app = QApplication([])

widgetMainWindow = QWidget()
widgetMainWindow.resize(500,500)

pushButton = QPushButton() # 创建按钮对象
pushButton.setParent(widgetMainWindow)
pushButton.resize(100,30) # 修改按钮大小

pushButton.setStyleSheet(
    """
    /* 按钮普通状态 */
    QPushButton{
        /* 倒角半径为50 */
        border-radius: 50; 
        /* 背景色 */
        background-color: blue;
    }
    
    /* 按钮停留状态 */
    QPushButton:hover{
        /* 倒角半径为50 */
        border-radius: 50; 
        /* 背景色 */
        background-color: green;
    }
    
    /* 按钮按下状态 */
    QPushButton:pressed{
        /* 倒角半径为50 */
        border-radius: 50; 
        /* 背景色 */
        background-color: red;
        /*左内边距为3像素，让按下时字向右移动3像素*/  
        padding-left:3px;
        /*上内边距为3像素，让按下时字向下移动3像素*/  
        padding-top:3px;
    }
    """)

widgetMainWindow.show()

app.exec_()

```cpp
//C++
pushButton->resize(100,30);
pushButton->setStyleSheet("\
            QPushButton{ \
                border-radius: 50; \
                background-color: blue;\
            }\
            QPushButton:hover{\
                border-radius: 50; \
                background-color: green;\
            }\
            QPushButton:pressed{\
                border-radius: 50; \
                background-color: red;\
                padding-left:3px;\
                padding-top:3px;\
            }\
        ");
```

按钮的三态效果主要是为了提升用户体现的，而此方法的实现也是通过样式表达到的。不同的是需要借助**选择器**，这与CCS里的相似，是基于控件的一些基本状态来限定样式表的规则，例如hover规则表示鼠标经过空间时的状态，而press表示按下按钮是的状态。

样式表的配置函数`setStyleSheet`的输入参数为一个字符串。上面展示了按钮三种状态下颜色的变化。其中`QPushButton{}`使用的是类型选择器，表示`{}`中的数据是`pushButton`中的QPushButton类型对象的参数，`QPushButton:pressed{}`中的`:pressed`为状态选择器，表示`{}`内的参数QPushButton对象的不同状态下的配置。

我们可以直接调用setStyleSheet函数，并增加2个按钮观察选择器的工作方式。

In [ ]:
from PyQt5.QtWidgets import QApplication, QWidget,QPushButton
from PyQt5.QtCore import Qt

app = QApplication([])

widgetMainWindow = QWidget()
widgetMainWindow.resize(500,500)

pushButton = QPushButton() # 创建按钮对象
pushButton.setParent(widgetMainWindow)
pushButton.resize(100,30) # 修改按钮大小
pushButton.setText("i am 1st")  # 增加文本显示

pushButtonTest = QPushButton() # 创建按钮对象
pushButtonTest.setParent(widgetMainWindow)
pushButtonTest.setGeometry(0, 50, 100, 30) # 修改按钮位置和大小
pushButtonTest.setText("i am 2nd") # 增加文本显示

widgetMainWindow.setStyleSheet(
    """
    /* 按钮普通状态 */
    QPushButton{
        /* 倒角半径为50 */
        border-radius: 50; 
        /* 背景色 */
        background-color: blue;
        /* 设置文本为白色 */
        color: white;
    }
    
    /* 按钮停留状态 */
    QPushButton:hover{
        /* 倒角半径为50 */
        border-radius: 50; 
        /* 背景色 */
        background-color: green;
        /* 设置文本为白色 */
        color: white;
    }
    
    /* 按钮按下状态 */
    QPushButton:pressed{
        /* 倒角半径为50 */
        border-radius: 50; 
        /* 背景色 */
        background-color: red;
        /*左内边距为3像素，让按下时字向右移动3像素*/  
        padding-left:3px;
        /*上内边距为3像素，让按下时字向下移动3像素*/  
        padding-top:3px;
        /* 设置文本为白色 */
        color: white;
    }
    """)

widgetMainWindow.show()

app.exec_()

上面代码中使用`widgetMainWindow.setStyleSheet`来配置样式表内容，其作用范围是widget及其子控件中的所有QPushButton对象。所以实例话的2个按钮对象都会用相同动作。`/* */`所包含的内容setStyleSheet函数将认为是注释不做解析。

## 按钮上显示图标

按钮上显示图标/图片改变的同样是按钮的样式，因此此功能同样需要用样式表实现。我们在圆形按钮基础上实现。图标需要用到png图片，那么为什么是png图片呢？
// todo


In [ ]:
from PyQt5.QtWidgets import QApplication, QWidget,QPushButton
from PyQt5.QtCore import Qt

app = QApplication([])

# 创建窗口用于放置按钮和标签
widgetMainWindow = QWidget()
widgetMainWindow.resize(300, 300)

pushButton = QPushButton()  
pushButton.setParent(widgetMainWindow)
pushButton.setGeometry(20, 70, 100, 100)

# 增加属性配置表设置代码
# Todo
# pushButtonIcon = ";"
pushButtonIcon = "border-image: url(./image/button_login.png);"
pushButton.setStyleSheet("""
                         border-radius: 50; 
                         """
                         + pushButtonIcon)

widgetMainWindow.show()

app.exec_()

设置按钮样式表的`border-image`属性即可在按钮上显示图标，显示发现按钮边界处图标和背景会有一定的交错，先简单处理不显示背景色

```
"""
border-radius: 50; 
"""
+ pushButtonIcon)
```

那么是什么原因导致了这条红边呢？这需要从QT的显示原理上分析了(todo)

# 捉迷藏小游戏开发

* 游戏有2个按钮，只有一个上面会显示"i am here"
* 当鼠标点击后文字会再次随机出现在其中一个。
* 当鼠标点击无文字按钮后，会出现miss，并且文字会再次随机出现在一个按钮上。

In [ ]:
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QLabel
from PyQt5.QtCore import Qt,QTime
import random

app = QApplication([])

widgetMainWindow = QWidget()
widgetMainWindow.resize(500,200)
# widgetMainWindow.setWindowFlags (Qt.FramelessWindowHint)
widgetMainWindow.setStyleSheet("background-color: black") #修改窗体背景颜色

创建游戏场地界面，然后需要创建游戏类，实现按钮和信号与槽机制。

In [ ]:
class ClickMeGame():
    def __init__(self,parent=None):
        self.txtPostion = 0 # 0-lef, 1-right
        # 创建1个标签
        self.stateLabel = QLabel()
        self.stateLabel.setGeometry(230, 50, 50, 50) # 设置按钮大小、按钮位置
        self.stateLabel.setStyleSheet("background: black; color: red")
        
        # 创建2个按钮
        self.pushButtonLeft = QPushButton()
        self.pushButtonLeft.setGeometry(100, 50, 100, 100) # 设置按钮大小、按钮位置
        self.pushButtonLeft.setStyleSheet("background: white; color: red")
        self.pushButtonLeft.setText("I am Here") # 设置按钮文本
        self.pushButtonLeft.clicked.connect(self.slot_left_click_handle)
        
        self.pushButtonRight = QPushButton()
        self.pushButtonRight.setGeometry(300, 50, 100, 100)
        self.pushButtonRight.setStyleSheet("background: white; color: red")
        self.pushButtonRight.setText("")
        self.pushButtonRight.clicked.connect(self.slot_right_click_handle)
        
        if parent is not None:
            self.pushButtonLeft.setParent(parent)
            self.pushButtonRight.setParent(parent)
            self.stateLabel.setParent(parent)
        
    def reflashButtonText(self):
        self.pushButtonLeft.setText("")
        self.pushButtonRight.setText("")
        if random.randint(0,99) > 50:
            self.pushButtonLeft.setText("I am Here")
            self.txtPostion = 0
        else:
            self.pushButtonRight.setText("I am Here")
            self.txtPostion = 1

    def slot_left_click_handle(self):
        if self.txtPostion != 0:
            self.stateLabel.setText("miss")
        else:
            self.stateLabel.setText("")
        self.reflashButtonText()
            
    def slot_right_click_handle(self):
        if self.txtPostion != 1:
            self.stateLabel.setText("miss")
        else:
            self.stateLabel.setText("")
        self.reflashButtonText()

In [ ]:
clickGame = ClickMeGame(widgetMainWindow)

widgetMainWindow.show()

app.exec_()

```cpp
//C++
主函数增加:
    stateLabel = new QLabel(this);
    stateLabel->setGeometry(230, 50, 50, 50);
    stateLabel->setStyleSheet("background: black; color: red");
    leftButton = new QPushButton(this);
    leftButton->setGeometry(100, 50, 100, 100);
    leftButton->setStyleSheet("background: white; color: red");
    leftButton->setText("I am Here");
    connect(leftButton,&QPushButton::clicked,this,&MainWindow::slot_left_click_handle);
    rightButton = new QPushButton(this);
    rightButton->setGeometry(300, 50, 100, 100);
    rightButton->setStyleSheet("background: white; color: red");
    rightButton->setText("");
    connect(rightButton,&QPushButton::clicked,this,&MainWindow::slot_right_click_handle);

    qsrand(QTime(0,0,0).secsTo(QTime::currentTime()));

增加如下函数:
void MainWindow::slot_left_click_handle()
{
    qDebug()<<"slot_left_click_handle";
    if(txtPostion!=0)
        stateLabel->setText("miss");
    else
        stateLabel->setText("");
    reflashButtonText();
}

void MainWindow::slot_right_click_handle()
{
    if(txtPostion!=1)
        stateLabel->setText("miss");
    else
        stateLabel->setText("");
    reflashButtonText();
}
void MainWindow::reflashButtonText()
{
    leftButton->setText("");
    rightButton->setText("");
    //qsrand(QTime(0,0,0).secsTo(QTime::currentTime()));
    qDebug()<<qrand()%99;
    if(qrand()%99 >50){
        leftButton->setText("I am Here");
        txtPostion=0;
    }else{
        rightButton->setText("I am Here");
        txtPostion=1;
    }
}
```

# 创建800x500界面

这里resize设置了窗口的大小，并使用`Qt.FramelessWindowHint`把标题栏隐藏起来。`FramelessWindowHint`属于Qt.Window类，用于产生一个无边框窗口。

In [ ]:
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton
from PyQt5.QtCore import Qt

app = QApplication([])

widgetMainWindow = QWidget()
widgetMainWindow.resize(800,500)
widgetMainWindow.setWindowFlags (Qt.FramelessWindowHint)
widgetMainWindow.setStyleSheet("background: black")

buttonLogIn = QPushButton() 
buttonLogIn.setParent(widgetMainWindow)
buttonLogIn.setGeometry(150, 200, 100, 100)
buttonLogIn.setStyleSheet("border-radius: 50; background-color: green; border-image: url(./image/button_login.png)")

buttonSysSet = QPushButton() 
buttonSysSet.setParent(widgetMainWindow)
buttonSysSet.setGeometry(350, 200, 100, 100)
buttonSysSet.setStyleSheet("border-radius: 50; background-color: green; border-image: url(./image/button_set.png)")

buttonReplay = QPushButton() 
buttonReplay.setParent(widgetMainWindow)
buttonReplay.setGeometry(550, 200, 100, 100)
buttonReplay.setStyleSheet("border-radius: 50; background-color: green; border-image: url(./image/button_replay.png)")

widgetMainWindow.show()

app.exec_()

# 任务：捉迷藏小游戏升级版开发

* 有2个按钮，只有一个按钮上显示“i am here”
* 鼠标点击按钮后文字消失，如果按错游戏结束
* 增加游戏开始按钮，单击后游戏开始计时，计数
* 游戏开始后进入30s倒计时，时间结束后游戏停止